In [55]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [56]:
### load the trained model, scaler pickle, one hot

model =load_model('model.h5')

# load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler =pickle.load(file)

In [57]:
# Sample data
import pandas as pd

input_data = {
    "CreditScore": [619, 608, 502, 750, 690, 710, 580, 720, 640, 795],
    "Geography": ["France", "Spain", "France", "Germany", "France", "Spain", "Germany", "France", "Spain", "France"],
    "Gender": ["Female", "Female", "Female", "Male", "Male", "Female", "Male", "Female", "Male", "Female"],
    "Age": [42, 41, 42, 29, 36, 45, 50, 38, 30, 27],
    "Tenure": [2, 1, 8, 3, 5, 7, 9, 6, 4, 2],
    "Balance": [0.00, 83807.86, 159660.80, 62000.55, 120450.12, 54000.33, 135000.45, 73000.20, 0.00, 48900.00],
    "NumOfProducts": [1, 1, 3, 2, 1, 2, 2, 1, 1, 2],
    "HasCrCard": [1, 0, 1, 1, 1, 0, 1, 0, 1, 1],
    "IsActiveMember": [1, 1, 0, 1, 0, 1, 0, 1, 1, 1],
    "EstimatedSalary": [101348.88, 112542.58, 113931.57, 95000.40, 88000.00, 122300.77, 77500.55, 108900.25, 56000.10, 130400.99]
}
input_data_df = pd.DataFrame(input_data)
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,750,Germany,Male,29,3,62000.55,2,1,1,95000.40
4,690,France,Male,36,5,120450.12,1,1,0,88000.00
5,710,Spain,Female,45,7,54000.33,2,0,1,122300.77
6,580,Germany,Male,50,9,135000.45,2,1,0,77500.55
7,720,France,Female,38,6,73000.20,1,0,1,108900.25
8,640,Spain,Male,30,4,0.00,1,1,1,56000.10
9,795,France,Female,27,2,48900.00,2,1,1,130400.99


In [58]:
# One-hot encoder 'Geography
import pandas as pd

# Then transform
geo_encoder = onehot_encoder_geo.transform(input_data_df[['Geography']]).toarray()

geo_encoder_df = pd.DataFrame(
    geo_encoder,
    columns=onehot_encoder_geo.get_feature_names_out()
)
geo_encoder_df

# Label encoding "Gender"
input_data_df['Gender']=label_encoder_gender.transform(input_data_df['Gender'])

In [59]:
# concatination 
input_data_df=pd.concat([input_data_df.drop(['Geography'],axis=1),geo_encoder_df],axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,750,1,29,3,62000.55,2,1,1,95000.40,0.0,1.0,0.0
4,690,1,36,5,120450.12,1,1,0,88000.00,1.0,0.0,0.0
5,710,0,45,7,54000.33,2,0,1,122300.77,0.0,0.0,1.0
6,580,1,50,9,135000.45,2,1,0,77500.55,0.0,1.0,0.0
7,720,0,38,6,73000.20,1,0,1,108900.25,1.0,0.0,0.0
8,640,1,30,4,0.00,1,1,1,56000.10,0.0,0.0,1.0
9,795,0,27,2,48900.00,2,1,1,130400.99,1.0,0.0,0.0


In [60]:
# scaling the input data
input_scaled=scaler.transform(input_data_df)
input_scaled

array([[-0.27700291, -1.09997489,  0.28778948, -1.03940009, -1.25599716,
        -0.89163935,  0.63367318,  0.95214374,  0.04564911,  0.98019606,
        -0.57581067, -0.56349184],
       [-0.38963278, -1.09997489,  0.1912806 , -1.38035469,  0.09315993,
        -0.89163935, -1.57810057,  0.95214374,  0.24046943, -1.02020406,
        -0.57581067,  1.77464858],
       [-1.47497511, -1.09997489,  0.28778948,  1.0063275 ,  1.314257  ,
         2.50185008,  0.63367318, -1.0502616 ,  0.26464405,  0.98019606,
        -0.57581067, -0.56349184],
       [ 1.06431638,  0.90911166, -0.96682594, -0.69844549, -0.25789887,
         0.80510537,  0.63367318,  0.95214374, -0.06484276, -1.02020406,
         1.73668197, -0.56349184],
       [ 0.44997167,  0.90911166, -0.2912638 , -0.0165363 ,  0.68303497,
        -0.89163935,  0.63367318, -1.0502616 , -0.18668095,  0.98019606,
        -0.57581067, -0.56349184],
       [ 0.65475324, -1.09997489,  0.57731611,  0.6653729 , -0.38668815,
         0.80510537, -

In [61]:
# prediction
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 67ms/step


array([[0.3105662 ],
       [0.19850576],
       [0.9908931 ],
       [0.00912655],
       [0.09426513],
       [0.07904557],
       [0.4879664 ],
       [0.10376216],
       [0.13914447],
       [0.0121731 ]], dtype=float32)

In [74]:
for i in range(0,len(prediction)):
    print(prediction[i][0])
    if prediction[i][0]>0.5:
        print('the customer is likely to churn. ')
    else:
        print('The customer is not likely to churn')

0.3105662
The customer is not likely to churn
0.19850576
The customer is not likely to churn
0.9908931
the customer is likely to churn. 
0.009126551
The customer is not likely to churn
0.094265126
The customer is not likely to churn
0.07904557
The customer is not likely to churn
0.4879664
The customer is not likely to churn
0.10376216
The customer is not likely to churn
0.13914447
The customer is not likely to churn
0.012173099
The customer is not likely to churn
